In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from mini_yolov3.model import MiniYOLOV3
from mini_yolov3.trainer import Trainer
import torch
from torchvision.transforms import v2
from mini_yolov3.dataset import SVHNDataset
from torch.utils.data import Subset

In [36]:
anchors = torch.Tensor([[0.2, 0.8]])

In [38]:
model = MiniYOLOV3(
    image_size=64,
    num_classes=10,
    anchors=anchors
)

model.to("mps")

MiniYOLOV3(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (3): Conv2d(32, 15, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
)

In [69]:
train_dataset = SVHNDataset(split="train", image_transform=v2.Compose([
                        v2.ToTensor(),
                        v2.Resize((64, 64))  
                      ]))
train_dataset = Subset(train_dataset, range(1))

/Users/ericchen/Library/Caches/pypoetry/virtualenvs/mini-yolov3-a3VRlvzj-py3.10/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [70]:
train_dataset[0]["bbox"]

tensor([[0.3551, 0.0217, 0.1963, 0.8696],
        [0.5327, 0.0652, 0.1495, 0.8696]])

In [73]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    num_epochs=100,
    lr=1e-5
)

In [74]:
trainer.train()

100%|██████████| 100/100 [00:04<00:00, 20.14it/s, loss=tensor(53.5012, device='mps:0', grad_fn=<AddBackward0>), coord_loss=tensor(48.0317, device='mps:0', grad_fn=<MulBackward0>), obj_conf_loss=tensor(2.1238, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), class_loss=tensor(1.4496, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), noobj_loss=tensor(1.8960, device='mps:0', grad_fn=<MulBackward0>)]


[53.21305847167969,
 53.22623825073242,
 53.2403678894043,
 53.255645751953125,
 53.270652770996094,
 53.276390075683594,
 53.27241134643555,
 53.26689529418945,
 53.26404571533203,
 53.26060485839844,
 53.25682067871094,
 53.25327682495117,
 53.25035858154297,
 53.2481575012207,
 53.24673080444336,
 53.245906829833984,
 53.24628829956055,
 53.24726104736328,
 53.24906539916992,
 53.25175857543945,
 53.25541687011719,
 53.25857925415039,
 53.26134490966797,
 53.265167236328125,
 53.27032470703125,
 53.27712631225586,
 53.28420639038086,
 53.291343688964844,
 53.297386169433594,
 53.303192138671875,
 53.3082160949707,
 53.31195831298828,
 53.31493377685547,
 53.31724548339844,
 53.31938552856445,
 53.32235336303711,
 53.32625961303711,
 53.33103561401367,
 53.3365478515625,
 53.3423957824707,
 53.348304748535156,
 53.35380172729492,
 53.35853958129883,
 53.362518310546875,
 53.36616134643555,
 53.37029266357422,
 53.375267028808594,
 53.380271911621094,
 53.38439178466797,
 53.387737274